In [1]:
from pathlib import Path

DATA = Path("/content/drive/MyDrive/랭체인 AI 영상객체탐지분석 플랫폼 구축/data")

print("DATA exists:", DATA.exists())
print("DATA:", DATA)

DATA exists: True
DATA: /content/drive/MyDrive/랭체인 AI 영상객체탐지분석 플랫폼 구축/data


In [2]:
# images 폴더 찾기
IMG_DIR = None
for cand in ["images", "image", "imgs"]:
    p = DATA / cand
    if p.exists():
        IMG_DIR = p
        break

In [3]:
# annotations 폴더 찾기 (annotations / annotation 둘 다 커버)
ANN_DIR = None
for cand in ["annotations", "annotation", "ann", "labels_xml"]:
    p = DATA / cand
    if p.exists():
        ANN_DIR = p
        break

print("IMG_DIR:", IMG_DIR)
print("ANN_DIR:", ANN_DIR)

IMG_DIR: /content/drive/MyDrive/랭체인 AI 영상객체탐지분석 플랫폼 구축/data/images
ANN_DIR: /content/drive/MyDrive/랭체인 AI 영상객체탐지분석 플랫폼 구축/data/annotations


In [4]:
# 파일 개수 확인
img_files = []
if IMG_DIR:
    for ext in ("*.jpg", "*.jpeg", "*.png"):
        img_files += list(IMG_DIR.glob(ext))

xml_files = []
if ANN_DIR:
    xml_files = list(ANN_DIR.glob("*.xml"))

print("num images:", len(img_files))
print("num xml   :", len(xml_files))
print("sample image:", img_files[0] if img_files else None)
print("sample xml  :", xml_files[0] if xml_files else None)

num images: 877
num xml   : 877
sample image: /content/drive/MyDrive/랭체인 AI 영상객체탐지분석 플랫폼 구축/data/images/road100.png
sample xml  : /content/drive/MyDrive/랭체인 AI 영상객체탐지분석 플랫폼 구축/data/annotations/road0.xml


In [5]:
from pathlib import Path
from collections import Counter

DATA = Path("/content/drive/MyDrive/랭체인 AI 영상객체탐지분석 플랫폿폼 구축/data")

img_exts = {".jpg", ".jpeg", ".png"}
img_paths, xml_paths = [], []

for p in DATA.rglob("*"):
    if p.is_file():
        if p.suffix.lower() in img_exts:
            img_paths.append(p)
        elif p.suffix.lower() == ".xml":
            xml_paths.append(p)

img_dir_top = Counter([p.parent for p in img_paths]).most_common(5)
xml_dir_top = Counter([p.parent for p in xml_paths]).most_common(5)

print("[Top image dirs]")
for d,c in img_dir_top:
    print(c, "->", d)

print("\n[Top xml dirs]")
for d,c in xml_dir_top:
    print(c, "->", d)


[Top image dirs]

[Top xml dirs]


In [ ]:
import random, shutil
from pathlib import Path
import xml.etree.ElementTree as ET
from PIL import Image

random.seed(42)

# 위에서 잡힌 IMG_DIR, ANN_DIR를 그대로 사용한다고 가정
# (만약 너가 강제 탐색으로 경로를 확인했다면 아래 두 줄을 직접 경로로 바꿔도 됨)
# IMG_DIR = Path(".../images")
# ANN_DIR = Path(".../annotations")

OUT = Path("/content/road_sign_yolo")
(OUT / "images/train").mkdir(parents=True, exist_ok=True)
(OUT / "images/val").mkdir(parents=True, exist_ok=True)
(OUT / "labels/train").mkdir(parents=True, exist_ok=True)
(OUT / "labels/val").mkdir(parents=True, exist_ok=True)

CLASSES = ["trafficlight", "stop", "speedlimit", "crosswalk"]
name_to_id = {n:i for i,n in enumerate(CLASSES)}

def norm_name(s: str) -> str:
    s = (s or "").strip().lower().replace(" ", "")
    if s in ["trafficlight", "traficlight", "trafficlights"]:
        return "trafficlight"
    if s in ["speedlimit", "speedlimitsign", "speedlimitsigns"]:
        return "speedlimit"
    if s in ["crosswalk", "crosswalksign", "pedcrossing", "pedestriancrossing"]:
        return "crosswalk"
    if s in ["stop", "stopsign", "stopsigns"]:
        return "stop"
    return s

def voc_to_yolo(xml_path: Path, img_w: int, img_h: int):
    tree = ET.parse(xml_path)
    root = tree.getroot()
    labels = []
    for obj in root.findall("object"):
        name = norm_name(obj.findtext("name"))
        if name not in name_to_id:
            continue
        bnd = obj.find("bndbox")
        if bnd is None:
            continue
        xmin = float(bnd.findtext("xmin"))
        ymin = float(bnd.findtext("ymin"))
        xmax = float(bnd.findtext("xmax"))
        ymax = float(bnd.findtext("ymax"))

        cx = ((xmin + xmax) / 2.0) / img_w
        cy = ((ymin + ymax) / 2.0) / img_h
        bw = (xmax - xmin) / img_w
        bh = (ymax - ymin) / img_h

        cx = min(max(cx, 0.0), 1.0)
        cy = min(max(cy, 0.0), 1.0)
        bw = min(max(bw, 0.0), 1.0)
        bh = min(max(bh, 0.0), 1.0)

        labels.append((name_to_id[name], cx, cy, bw, bh))
    return labels

img_files = sorted(list(IMG_DIR.glob("*.jpg")) + list(IMG_DIR.glob("*.jpeg")) + list(IMG_DIR.glob("*.png")))
if not img_files:
    raise RuntimeError("이미지 파일이 안 보여. IMG_DIR 경로를 다시 확인해줘.")

idx = list(range(len(img_files)))
random.shuffle(idx)
val_n = int(len(img_files) * 0.2)
val_set = set(idx[:val_n])

missing_xml = 0
written = {"train": 0, "val": 0}

for i, img_path in enumerate(img_files):
    split = "val" if i in val_set else "train"
    stem = img_path.stem
    xml_path = ANN_DIR / f"{stem}.xml"
    if not xml_path.exists():
        missing_xml += 1
        continue

    with Image.open(img_path) as im:
        w, h = im.size

    yolo_labels = voc_to_yolo(xml_path, w, h)

    shutil.copy2(img_path, OUT / f"images/{split}" / img_path.name)

    label_path = OUT / f"labels/{split}" / f"{stem}.txt"
    with open(label_path, "w") as f:
        for c, cx, cy, bw, bh in yolo_labels:
            f.write(f"{c} {cx:.6f} {cy:.6f} {bw:.6f} {bh:.6f}\n")

    written[split] += 1

print("DONE")
print("written:", written)
print("missing_xml:", missing_xml)
print("OUT:", OUT)


In [ ]:
import random, shutil
from pathlib import Path
import xml.etree.ElementTree as ET
from PIL import Image

random.seed(42)

# 위에서 잡힌 IMG_DIR, ANN_DIR를 그대로 사용한다고 가정
# (만약 너가 강제 탐색으로 경로를 확인했다면 아래 두 줄을 직접 경로로 바꿔도 됨)
# IMG_DIR = Path(".../images")
# ANN_DIR = Path(".../annotations")

OUT = Path("/content/road_sign_yolo")
(OUT / "images/train").mkdir(parents=True, exist_ok=True)
(OUT / "images/val").mkdir(parents=True, exist_ok=True)
(OUT / "labels/train").mkdir(parents=True, exist_ok=True)
(OUT / "labels/val").mkdir(parents=True, exist_ok=True)

CLASSES = ["trafficlight", "stop", "speedlimit", "crosswalk"]
name_to_id = {n:i for i,n in enumerate(CLASSES)}

def norm_name(s: str) -> str:
    s = (s or "").strip().lower().replace(" ", "")
    if s in ["trafficlight", "traficlight", "trafficlights"]:
        return "trafficlight"
    if s in ["speedlimit", "speedlimitsign", "speedlimitsigns"]:
        return "speedlimit"
    if s in ["crosswalk", "crosswalksign", "pedcrossing", "pedestriancrossing"]:
        return "crosswalk"
    if s in ["stop", "stopsign", "stopsigns"]:
        return "stop"
    return s

def voc_to_yolo(xml_path: Path, img_w: int, img_h: int):
    tree = ET.parse(xml_path)
    root = tree.getroot()
    labels = []
    for obj in root.findall("object"):
        name = norm_name(obj.findtext("name"))
        if name not in name_to_id:
            continue
        bnd = obj.find("bndbox")
        if bnd is None:
            continue
        xmin = float(bnd.findtext("xmin"))
        ymin = float(bnd.findtext("ymin"))
        xmax = float(bnd.findtext("xmax"))
        ymax = float(bnd.findtext("ymax"))

        cx = ((xmin + xmax) / 2.0) / img_w
        cy = ((ymin + ymax) / 2.0) / img_h
        bw = (xmax - xmin) / img_w
        bh = (ymax - ymin) / img_h

        cx = min(max(cx, 0.0), 1.0)
        cy = min(max(cy, 0.0), 1.0)
        bw = min(max(bw, 0.0), 1.0)
        bh = min(max(bh, 0.0), 1.0)

        labels.append((name_to_id[name], cx, cy, bw, bh))
    return labels

img_files = sorted(list(IMG_DIR.glob("*.jpg")) + list(IMG_DIR.glob("*.jpeg")) + list(IMG_DIR.glob("*.png")))
if not img_files:
    raise RuntimeError("이미지 파일이 안 보여. IMG_DIR 경로를 다시 확인해줘.")

idx = list(range(len(img_files)))
random.shuffle(idx)
val_n = int(len(img_files) * 0.2)
val_set = set(idx[:val_n])

missing_xml = 0
written = {"train": 0, "val": 0}

for i, img_path in enumerate(img_files):
    split = "val" if i in val_set else "train"
    stem = img_path.stem
    xml_path = ANN_DIR / f"{stem}.xml"
    if not xml_path.exists():
        missing_xml += 1
        continue

    with Image.open(img_path) as im:
        w, h = im.size

    yolo_labels = voc_to_yolo(xml_path, w, h)

    shutil.copy2(img_path, OUT / f"images/{split}" / img_path.name)

    label_path = OUT / f"labels/{split}" / f"{stem}.txt"
    with open(label_path, "w") as f:
        for c, cx, cy, bw, bh in yolo_labels:
            f.write(f"{c} {cx:.6f} {cy:.6f} {bw:.6f} {bh:.6f}\n")

    written[split] += 1

print("DONE")
print("written:", written)
print("missing_xml:", missing_xml)
print("OUT:", OUT)


In [ ]:
from pathlib import Path

OUT = Path("/content/road_sign_yolo")
yaml_text = f"""path: {OUT}
train: images/train
val: images/val

nc: 4
names: [trafficlight, stop, speedlimit, crosswalk]
"""
Path("/content/data.yaml").write_text(yaml_text, encoding="utf-8")
print(yaml_text)


In [ ]:
!pip -q install ultralytics
!yolo detect train data=/content/data.yaml model=yolov8n.pt epochs=50 imgsz=640 batch=16 project=/content/runs name=roadsign


In [ ]:
!yolo detect predict model=/content/runs/roadsign/weights/best.pt source=/content/road_sign_yolo/images/val save=True conf=0.25 project=/content/runs name=pred_roadsign
